In [4]:
# Kütüphaneler
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [5]:

# Veri setini yükleyin
df = pd.read_csv("kc_house_data.csv")

# Veri setinin boyutunu kontrol edin
print(f"Veri seti boyutu: {df.shape}")

# İlk birkaç satırı inceleyin
print(df.head())

# Özet istatistikler
print(df.describe())

# Eksik değerleri kontrol edin
print(f"Eksik değerler: {df.isnull().sum().sum()}")


Veri seti boyutu: (21613, 21)
           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4      8080     1.0           0     0  ...      8        1680              0   

   yr_built  yr_reno